## Abstract  

我们提出了一个概念上简单，灵活，通用的对象实例分割框架。我们的方法有效地检测图像中的对象，同时为每个实例生成高质量的分割掩膜。该方法被称为 Mask R-CNN，通过增加一个用于预测对象掩码的分支，并与现有的用于边界框识别的分支相结合，来扩展 faster R-CNN。Mask R-CNN 训练简单，以5 fps的速度运行,相比 faster R-CNN 只增加了一小部分的运算时间。此外，Mask R-CNN 很容易推广到其他任务，例如允许我们在相同的框架下估计人的姿势。我们在 COCO 的三个数据集上都表现出了 state of art 的效果，包括实例分割，边界框对象检测和人体关键点检测。在没有任何技巧的情况下， Mask R-CNN 的效果都由于目前所有在单个任务上的方法，包括 COCO 2016 挑战的获胜者。希望我们简单而有效的方法能够作为一个 base line，并有助于简化未来在实例识别方面的研究。

## 1. Introduction  

视觉社区在短时间内迅速提高了目标检测和语义分割的效果。在很大程度上，这些进步是由强大的 base line 系统驱动的，例如分别用于目标检测和语义分割的 faster RCNN 和全卷积网络（FCN）框架。这些方法在概念上是直观的，并提供了灵活性和健壮性，以及快速的训练和预测时间。我们在这项工作中的目标是开发一个可比较的实例分割框架。  

实例分割是具有挑战性的，因为它需要正确检测图像中的所有对象，同时精确地分割每个实例。 因此，它将经典的计算机视觉任务中的目标检测和语义分割结合起来，目标检测的目标是对各个对象进行分类，并使用边界框对定位出每个目标的位置，语义分割的目标是在不区分每个实例的情况下将每个像素分类为固定的一组类别。鉴于此，可能需要一个复杂的方法才能获得好的结果。 然而，我们表明，一个令人惊讶的简单、灵活、快速的系统可以超越先前的实例分割结果。  

我们的方法称为 Mask R-CNN，通过在现有的分类和边界框回归上并行的在每个感兴趣区域（RoI）上添加一个用于预测分割掩码的分支来扩展 Faster R-CNN。掩模分支是应用于每个 ROI 的小 FCN，以 pixel-to-pixel 的方式预测分割掩模。基于 Faster R-CNN 框架，可以很方便的 构建 mask R-CNN 框架。另外，掩码分支只增加了一个小的计算开销，实现了一个快速的系统并可以进行快速的实验。  

Mask R-CNN 是 Faster R-CNN 的一个直观的扩展，但是正确地构建掩模分支对于获得好的结果是至关重要的。最重要的是，Faster R-CNN 并不是针对网络输入和输出之间的像素对像素的转换而设计的。最明显的是 RoIPool 为特征提取执行粗略的空间量化。为了解决这个错位问题，我们提出了一个简单的，无量化的 RoIAlign 层，保留了精确的空间位置信息。尽管是一个看似微小的变化，但 RoIlign 具有很大的影响力：它将掩模精度提高了10％到50％，在更严格的定位指标下显示出更大的收益。其次，我们发现将掩码和类别预测分离是非常重要的：我们独立预测每个类别的二进制掩码，而不需要在类别间进行竞争，并且依靠网络的 ROI 分类分支来预测类别。相比之下，FCNs 通常结合分割和分类执行每像素多类别分类，并且在我们的实验中这种方法在实例分割方面表现不佳。  

Mask R-CNN 在 COCO 实例分割任务中超越了之前所有先进的 single-model 结果。作为副产品，我们的方法也擅长 COCO 物体检测任务。在对照实验中，我们评估了多个基本实例，这使我们能够展示它的稳健性并分析核心因素的影响。  

我们的模型可以在 GPU 上以每帧大约 200ms 的速度运行，并且在单个 8-GPU 计算机上对 COCO 进行一到两天的训练。我们相信，快速训练和测试速度，以及灵活性和准确性的框架，将有益于并且简化未来对实例分割的研究。  

最后，我们通过 COCO 关键点数据集上的人体姿态估计任务展示了我们框架的通用性。通过将每个关键点视为一个单独的二进制掩码，只需最小的修改即可应用 Mask R-CNN 来检测特定实例的姿势。在没用任何技巧的情况下，Mask R-CNN超越了 2016 年 COCO 竞赛的胜利者，同时以5 fps的速度运行。因此Mask R-CNN 可以被更广泛地看作是实例级识别的灵活框架，并且可以容易地扩展到更复杂的任务。  

## 2. Related Work  

R-CNN：基于区域的 CNN（R-CNN）方法对边界框目标进行检测是为了管理候选目标的数量，并在每个ROI上独立评估卷积网络。R-CNN 允许在使用 RoIPool 的特征地图上参与 RoI，从而加快速度和精度。Fast R-CNN 通过学习区域建议网络（RPN）的注意机制来推进这一流程。Faster R-CNN 对于许多后续改进具有灵活性和可靠性，并且是当前在几个基准测试中主要的框架。  

Instance Segmentation：在RCNN有效性的驱动下，许多实例分割的方法都基于 segment proposals。早期的方法采用了自下而上的方法。DeepMask 和后续的作品学习提出候选分割区域，然后由 Fast R-CNN 进行分类。在这些方法中，分割先于识别，这是比较慢并且不太准确。 同样，戴等人提出了一个复杂的多阶段级联，先从 bounding-box proposals 中预测 segment proposals，然后进行分类。相反，我们的方法是基于掩码和类标签并行预测的，更简单灵活。  

最近，李等将 segment proposal system 和 object detection system 合并为“全卷积实例分割”（FCIS）。思想是全卷积预测一组位置敏感的输出通道。 这些通道同时进行分类、检测和分割，使系统更快。但 FCIS 在重叠实例上表现出系统性的错误并产生虚假边缘，表明它受到了分割实例的根本困难的挑战。

## 3. Mask R-CNN  

Mask R-CNN 在概念上是简单的：Faster R-CNN 对于每个候选对象具有两个输出，一个类别标签和一个边界框偏移;为此，我们添加一个输出对象掩码的第三个分支。因此 Mask R-CNN 是一个自然而直观的想法。但是 Mask 输出与分类和检测输出不同，需要提取更精细的目标空间布局。接下来，我们介绍 Mask R-CNN 的关键点，包括像素到像素的对齐，这是 Fast / Raster R-CNN 中没有的部分。  

Faster R-CNN：我们首先简要回顾一下 Faster R-CNN。 Faster R-CNN 由两个阶段组成。第一阶段称为区域提议网络（RPN），提出候选目标边界框。第二阶段，本质上是 Fast R-CNN，使用 RoIPool 从每个候选框中提取特征，并执行分类和边界框回归。这两部分可以共享权重，从而可以加快前向运行速度。  


Mask R-CNN：Mask R-CNN 采用相同的两阶段流程，第一阶段相同（即RPN）。在第二阶段，与预测类和边框偏移并行，Mask R-CNN 为每个 RoI 输出一个二进制掩码。这与最近的分类取决于 mask 预测的系统形成对比。我们的方法遵循 Faster R-CNN 的思想，并行地应用了边界框分类和回归。  

形式上，在训练期间，我们将每个抽样的RoI的多任务丢失定义为L = Lcls + Lbox + Lmask。 分类损失Lcls和边界框丢失Lbox与[12]中定义的相同。 掩码分支对于每个RoI具有Km2维输出，其编码分辨率为m×m的K个二进制掩码，每个K个类别一个掩码。 为此，我们应用每像素S形，并将Lmask定义为平均二叉交叉熵损失。 对于与地面真值等级k相关的RoI，Lmask仅在第k个掩模上定义（其他掩模输出不会造成损失）。  

我们对Lmask的定义允许网络为每个班级生成口罩，而不需要在班级间进行竞争。 我们依靠专用分类分支来预测用于选择输出掩码的类别标签。 这个解掩码和类别预测。 这与将FCN应用于语义分割时的常见做法不同[29]，语义分割通常使用每像素softmax和多项式交叉熵损失。 在这种情况下，跨班的面具竞争; 在我们的情况下，每像素S形和二元损失，他们不。 我们通过实验显示，这个公式是实例分割结果的关键。  

掩码表示法：掩码编码输入对象的空间布局。 因此，与通过完全连接（fc）层不可避免地折叠成短输出矢量的类标签或框偏移不同，提取掩码的空间结构可以通过卷积提供的像素到像素的对应来自然地解决。  

具体而言，我们使用FCN预测每个RoI的m×m掩码[29]。 这允许掩码分支中的每个层保持显式的m×m对象空间布局而不将其折叠成缺少空间维度的向量表示。 不同于之前的方法，fc层用于掩模预测[32,33,10]，我们的完全卷积表示需要更少的参数，并且如实验所证明的那样更准确。  

这种像素到像素行为要求我们的RoI特征（其本身是小特征图）能够很好地对齐以忠实地保留显式的每像素空间对应。 这促使我们开发了以下RoAlign层，在掩码预测中扮演着重要的角色。  

RoIlign：RoIPool [12]是从每个RoI提取小特征映射（例如7×7）的标准操作。 RoIPool首先将浮点数RoI量化为特征映射的离散粒度，然后将这个量化的RoI细分为自身量化的空间仓，最后聚合每个仓所涵盖的特征值（通常通过最大池）。 通过计算[x / 16]在例如连续坐标x上执行量化，其中16是特征映射步长，[·]是舍入; 同样，当分成分箱（例如，7×7）时执行量化。 这些量化引入了RoI和提取的特征之间的错位。 虽然这可能不会影响分类，这对于小的翻译是强健的，但是对于预测像素精确的掩码有很大的负面影响。  

为了解决这个问题，我们提出一个RoIlign层，去除RoIPool的严格量化，正确对齐提取的特征和输入。 我们提出的改变很简单：我们避免任何RoI边界或分组的量化（即我们使用x / 16而不是[x / 16]）。 我们使用双线性插值[22]来计算每个RoI bin中四个规则采样位置的输入特征值的精确值，并将结果汇总（使用最大值或平均值）。  

我们在§4.2中展示RoIlign会带来很大的改进。 我们也比较了[10]中提出的RoIWarp操作。 与RoIlign不同的是，RoIWarp忽略了对齐问题，并且在[10]中被实现为与RoIPool一样的量化RoI。 因此，即使RoIWarp也采用[22]所推荐的双线性重采样方法，如实验所示（表2c中更多细节），它与RoIPool相当，表明了对齐的关键作用。  

网络架构：为了演示我们的方法的一般性，我们实例化具有多种体系结构的Mask R-CNN。 为了清晰起见，我们区分：（i）用于整个图像上的特征提取的卷积骨架体系结构，以及（ii）分别应用于每个RoI的边界框识别（分类和回归）和掩模预测的网络头。  

我们用命名网络深度特征来表示骨干架构。 我们评估深度50或101层的ResNet [19]和ResNeXt [40]网络。 更快的R-CNN与ResNets [19]的原始实现从第四阶段的最终卷积层提取特征，我们称之为C4。 例如，ResNet-50的骨干网由ResNet-50-C4表示。 这是[19,10,21,36]中常用的选择。  

我们还探索了Lin等人最近提出的另一个更有效的骨干。 [27]，称为特征金字塔网络（FPN）。 FPN使用具有横向连接的自顶向下的体系结构来从单一比例输入构建网络内的特征金字塔。 更快的R-CNN和FPN骨干网根据其规模从不同层次的特征金字塔中提取RoI特征，否则其余的方法类似于香草ResNet。 使用ResNet-FPN骨干网进行MaskNRCNN特征提取，在精确度和速度方面都有非常好的收益。 有关FPN的更多细节，请参阅[27]。  

对于网络负责人，我们密切关注以前工作中提出的架构，我们添加了完全卷积掩码预测分支。 具体而言，我们从ResNet [19]和FPN [27]论文中扩展了更快的R-CNN盒头。 详细情况如图3所示。ResNet-C4主干上包含ResNet的第5级（即9层'res5'[19]），它是计算密集型的。 对于FPN，骨干已经包含了res5，因此允许使用更少的滤波器的更高效的头部。  

我们注意到我们的面具分支有一个简单的结构。 更复杂的设计有改善性能的潜力，但不是这项工作的重点。  

### 3.1. Implementation Details  

我们在现有的快速/更快的R-CNN工作之后设置超参数[12,34,27]。 虽然这些决策是在原始文件中进行对象检测[12,34,27]，我们发现我们的实例分割系统对他们是健壮的。  

培训：和快速R-CNN一样，如果IoU的地面实况框至少为0.5，则RoI被认为是肯定的，否则为负。 掩模丢失Lmask仅在正RoI上定义。 掩码目标是RoI与其相关的地面实况蒙版之间的交集。  

我们采用图像中心训练[12]。 调整图像的大小以使其比例（较短边缘）为800像素[27]。 每个微型批次每个GPU有两个图像，每个图像有N个采样的RoI，比例为1：3的正负面[12]。 C4骨架的N为64（如[12,34]），FPN为512（如[27]）。 我们在8个GPU（有效小批量大小为16）上进行160k次迭代训练，学习率为0.02，在120k迭代时减少10。 我们使用0.0001的重量衰减和0.9的动量。  

RPN锚点跨越5个尺度和3个纵横比，在[27]之后。 为了方便消融，除非另有规定，否则RPN是单独训练的，不与Mask R-CNN共享特征。 对于本文中的每个条目，RPN和Mask R-CNN具有相同的骨干，因此它们是可共享的。  

推论：在测试时间，C4主干的提案编号为300（如[34]），FPN的提案编号为1000（如[27]）。 我们对这些提议运行盒子预测分支，然后是非最大抑制[14]。 然后将掩码分支应用到最高得分的100个检测框。 虽然这不同于训练中使用的并行计算，但是它加快了推理的速度，并提高了准确性（由于使用更少，更准确的RoI）。 掩码分支可以预测每个RoI的K个掩码，但是我们只使用第k个掩码，其中k是分类分支预测的类别。 然后将m×m浮点数掩码输出调整为RoI大小，并在阈值0.5下进行二进制化。  

请注意，由于我们只计算前100个检测框中的掩码，因此掩码R-CNN为其更快的R-CNN对象（例如，典型模型上的约20％）添加了边际运行时间。  